# Aquisition RMN
## Functions
Bloc to Run everytime :


In [3]:
%matplotlib qt
#to open matplotlib window

import csv
import matplotlib.pyplot as plt
import numpy as np
import datetime
import paramiko
import os
import time
from scipy import signal
import datetime



SFTP_PORT = 22
SFTP_USERNAME = "root"
SFTP_PASSWORD = "root"
REMOTE_PATH = "Pitaya-Tests/" 
REMOTE_FOLDER = "Pitaya-Tests"
SAMPLING_RATE = 125e+6

def run_acquisition_command(hostname, samplesNb, dec, 
                            FidNb, FileName, larmorFrequency, excitationDuration, delayRepeat): #voir si file path marche
    port = 22
    username = SFTP_USERNAME
    password = SFTP_PASSWORD
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    filePath = "mesures/" + FileName

    try:
        client.connect(hostname=hostname, username=username, password=password, port=port)          

        print(f"[INFO] Connexion établie à {hostname}")
        command = f"cd {REMOTE_FOLDER} && ./Acquisition_axi.exe {samplesNb} {dec} {FidNb} {filePath} {larmorFrequency} {excitationDuration} {delayRepeat}"
        stdin, stdout, stderr = client.exec_command(command)
        print(f"[CMD-SSH] {command}")
        output = stdout.read().decode()
        errors = stderr.read().decode()

        if output:
            print("[OUTPUT SSH]\n", output)
        if errors:
            print("[ERROR SHH]\n", errors)
        print("[INFO] Connexion SSH fermée.")

    except Exception as e:
        print(f"[ERREUR] SSH : {e}")

    finally:
        
        client.close()

def run_find_excitation_time_command(hostname, samplesNb, dec, 
                            FidNb, FileName, larmorFrequency, start_excitationDuration, step, wait_time):
    port = 22
    username = SFTP_USERNAME
    password = SFTP_PASSWORD
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    filePath = "balayage/" + FileName

    try:
        client.connect(hostname=hostname, username=username, password=password, port=port)          

        print(f"[INFO] Connexion établie à {hostname}")
        command = f"cd {REMOTE_FOLDER} && ./Find_excitation_time.exe {samplesNb} {dec} {FidNb} {filePath} {larmorFrequency} {start_excitationDuration} {step} {wait_time} &"
        stdin, stdout, stderr = client.exec_command(command)
        output = stdout.read().decode()
        errors = stderr.read().decode()

    except Exception as e:
        print(f"[ERREUR] SSH : {e}")

    finally:
        

        client.close()
        print(f"[CMD-SSH] {command}")
        if output:
            print("[OUTPUT SSH]\n", output)
        if errors:
            print("[ERROR SHH]\n", errors)
        print("[INFO] Connexion SSH fermée.")

def run_find_frequency_command(hostname, samplesNb, dec, 
                            FidNb, FileName, larmorFrequency, start_excitationDuration, step, wait_time):
    port = 22
    username = SFTP_USERNAME
    password = SFTP_PASSWORD
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    filePath = "balayage/" + FileName

    try:
        client.connect(hostname=hostname, username=username, password=password, port=port)          

        print(f"[INFO] Connexion établie à {hostname}")
        command = f"cd {REMOTE_FOLDER} && ./Find_Frequency.exe {samplesNb} {dec} {FidNb} {filePath} {larmorFrequency} {start_excitationDuration} {step} {wait_time}"
        stdin, stdout, stderr = client.exec_command(command)
        print(f"[CMD-SSH] {command}")
        output = stdout.read().decode()
        errors = stderr.read().decode()

    except Exception as e:
        print(f"[ERREUR] SSH : {e}")

    finally:
        
        client.close()
        
        if output:
            print("[OUTPUT SSH]\n", output)
        if errors:
            print("[ERROR SHH]\n", errors)
        print("[INFO] Connexion SSH fermée.")

def telecharger_fichier_sftp(nom_Fichier,folder_name, SFTP_HOST):
    """Télécharge le fichier CSV via SFTP"""
    
    try:
        # Connexion SFTP
        transport = paramiko.Transport((SFTP_HOST, SFTP_PORT))
        transport.connect(username=SFTP_USERNAME, password=SFTP_PASSWORD)
        sftp = paramiko.SFTPClient.from_transport(transport)
        
        print(f"Connexion SFTP établie avec {SFTP_HOST}")

        # Créer le dossier local avec timestamp
        now = datetime.datetime.now()
        nom_dossier_local = f"mesures/{folder_name}-{now.strftime('%Y%m%d_%H%M%S')}"
        os.makedirs(nom_dossier_local, exist_ok=True)
        
        # Liste des fichiers à télécharger
        fichier_distant = nom_Fichier
        
        # Téléchargement du fichier
        chemin_distant = REMOTE_PATH + folder_name+'/' + fichier_distant
        chemin_local = os.path.join(nom_dossier_local, fichier_distant)
        
        try:
            sftp.get(chemin_distant, chemin_local)
            fichier_local = chemin_local
            print(f"Téléchargé: {fichier_distant}")
        except FileNotFoundError:
            print(f"Fichier non trouvé: {chemin_distant}")
        except Exception as e:
            print(f"Erreur lors du téléchargement de {fichier_distant}: {e}")
    
        # Fermer la connexion
        sftp.close()
        transport.close()
        print("Connexion SFTP fermée")
        
        return fichier_local
        
    except Exception as e:
        print(f"Erreur de connexion SFTP: {e}")
        return ""

def open_file(pathFile_csv):
        voltage_acc = []

        with open(pathFile_csv, 'r', encoding='utf-8') as fichier_:
            lecteur = csv.reader(fichier_)

            # Lire et parser l'entête du fichier
            ligne_entete = next(lecteur)
            dsize = int(ligne_entete[0])
            decimation = int(ligne_entete[1])
            nombre_de_FID = int(ligne_entete[2])
            gain = float(ligne_entete[3])
            offset = float(ligne_entete[4])
            nb_bits = int(ligne_entete[5])

            # Initialisation
            voltage = [[] for _ in range(nombre_de_FID)]
            mean = []
            voltage_acc = np.zeros(dsize)  # Initialiser au bon format et taille

            # Lecture et conversion des tensions
            for j in range(nombre_de_FID):
                ligne = next(lecteur)

                signal = []

                # for val_bin in ligne:
                #     #val_bin = int(val_bin)
                #     val = convert_to_volt(val_bin, nb_bits, gain, offset)
                #     signal.append(val)  # pas besoin de np.append ici


                for val in ligne:
                    signal.append(float(val))

                # Convertir la ligne en tableau numpy
                signal = np.array(signal)

                #### Calcul de la moyenne et centrage
                #moyenne = np.mean(signal)
                #signal_centre = signal - moyenne

                # Stocker les données
                voltage[j] = signal
                #mean.append(moyenne)

                # Accumulation du signal total
                voltage_acc += signal

        # Création du tableau temps
        duree_mesure = (dsize * decimation) / SAMPLING_RATE 
        time = np.linspace(0, duree_mesure, dsize, endpoint=False)

        # Calcul de la moyenne et centrage du signal accumulé
        moyenne = np.mean(voltage_acc[int((dsize*0.5)):int((dsize*0.98))])
        voltage_acc = voltage_acc - moyenne

        print(f"Fichier {pathFile_csv} lu. {len(voltage)} signaux FID chargés.")
        return time, voltage, voltage_acc
    
def plot_acc(graph_name, time_axis, voltage_matrix, voltage_accumulated_axis):
    
    amountFID = len(voltage_matrix)

    plt.figure(figsize=(12, 7))

    # Affichage des courbes FID
    for w in range(amountFID):
        plt.plot(time_axis, voltage_matrix[w], marker='+', linestyle='-', label=f'FID {w+1}')

    # Affichage du signal accumulé
    ###############################
    ####----> plt.plot(time_axis, voltage_accumulated_axis, marker='+', linestyle='-', label='Total', linewidth=2, color='black')
    ##############################
    
    # Mise en forme du graphique
    plt.title(f'Superposition - {graph_name}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Tension (V)')
    plt.grid(False, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    #plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

def plot_acc_only(graph_name, time_axis, voltage_matrix, voltage_accumulated_axis):
    
    amountFID = len(voltage_matrix)

    plt.figure(figsize=(12, 7))

    # Affichage du signal accumulé
    plt.plot(time_axis, voltage_accumulated_axis, marker='+', linestyle='-', label='Total', linewidth=2, color='black')

    # Mise en forme du graphique
    plt.title(f'Accumulation de {amountFID}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Tension (V)')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    #plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

def subpolts_acc(graph_name, time_axis, voltage_matrix, voltage_accumulated_axis):
    # Supposons que voltage_matrix et time_axis soient déjà définis
    amountFID = len(voltage_matrix)

    # Créer une figure et un ensemble de sous-graphes
    fig, axs = plt.subplots(amountFID, 1, figsize=(12, 7 * amountFID), sharex=True)

    # Affichage des courbes FID sur des sous-graphes séparés
    for w in range(amountFID):
        axs[w].plot(time_axis, voltage_matrix[w], marker='+', linestyle='-', label=f'FID {w+1}')
        axs[w].set_title(f'FID {w+1}')
        axs[w].set_ylabel('Tension (V)')
        axs[w].grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
        axs[w].legend(loc='upper right')

    # Mise en forme globale du graphique
    fig.suptitle(f'Superposition - {graph_name}', y=1.02)  # y est légèrement ajusté pour éviter le chevauchement
    plt.xlabel('Temps (s)')
    plt.tight_layout()
    plt.show()

def plot_fourier_transform(time, voltage):
    # Ensure time and voltage are numpy arrays
    time = np.array(time)
    voltage = np.array(voltage)

    # Sampling interval and frequency
    dt = time[1] - time[0]
    fs = 1 / dt

    # Compute FFT
    N = len(voltage)
    fft_values = np.fft.fft(voltage)
    freq = np.fft.fftfreq(N, dt)

    # Keep only the positive frequencies
    #idx = np.where(freq >= 0)
    #freq = freq[idx]
    magnitude = np.abs(fft_values) * 2 / N  # Normalize amplitude

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(freq, magnitude)
    plt.title("Fourier Transform")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Amplitude")
    plt.legend()
    plt.grid(True, which='both')
    plt.minorticks_on()
    plt.grid(which='minor', alpha=0.2)
    plt.grid(which='major', alpha=0.5)
    #plt.tight_layout()
    plt.show()

## Run program
Open the Redpitaya webpage before running : [rp-f0d66d.local](http://rp-f0d66d.local/)

In [30]:
hostName = "169.254.29.18"
#===== PARAMETERS ======

sample_Amount = 2720              #Number of points measured (MAX = 524288) must be a multiple of 2
decimation = 1024                    #Decimation
aquisiton_Amount = 10         #Nubmer of acquisitons
experiment_name = "mesure_antenne_aimant.csv"            #name of the file of the measured tension
larmor_Frequency_Hertz = 13255000.000000          #Larmor frequency
excitation_duration_seconds = 40.000e-6     #Excitation time
delay_repeat_seconds = 5


total_time = (sample_Amount * decimation)/SAMPLING_RATE
print(f"temps mesuré : {total_time}s")
temps_secondes = (total_time+delay_repeat_seconds)*aquisiton_Amount+3
#print(f"temps total secondes: {temps_secondes}")

print(f"temps total :"+str(datetime.timedelta(seconds=temps_secondes)))

nb_cycles = larmor_Frequency_Hertz*excitation_duration_seconds
print(f"nb cycles burst : {nb_cycles}")
print(sample_Amount)

temps mesuré : 0.02228224s
temps total :0:00:53.222822
nb cycles burst : 530.2
2720


In [31]:
run_acquisition_command(hostName,
                        sample_Amount, 
                        decimation, 
                        aquisiton_Amount, 
                        experiment_name, 
                        larmor_Frequency_Hertz, 
                        excitation_duration_seconds,
                        delay_repeat_seconds)


[INFO] Connexion établie à 169.254.29.18
[CMD-SSH] cd Pitaya-Tests && ./Acquisition_axi.exe 2720 1024 10 mesures/mesure_antenne_aimant.csv 13255000.0 4e-05 5
[OUTPUT SSH]
 larmor 13255000.000000, duration excitation 0.000040
fichier crée : mesures/mesure_antenne_aimant.csv
wait to be filled
Tr pos1: 0x0
ecriture FID 0
wait to be filled
Tr pos1: 0x0
ecriture FID 1
wait to be filled
Tr pos1: 0x0
ecriture FID 2
wait to be filled
Tr pos1: 0x0
ecriture FID 3
wait to be filled
Tr pos1: 0x0
ecriture FID 4
wait to be filled
Tr pos1: 0x0
ecriture FID 5
wait to be filled
Tr pos1: 0x0
ecriture FID 6
wait to be filled
Tr pos1: 0x0
ecriture FID 7
wait to be filled
Tr pos1: 0x0
ecriture FID 8
wait to be filled
Tr pos1: 0x0
ecriture FID 9
Temps d'execution : 0.303249

[INFO] Connexion SSH fermée.


In [32]:
pathFile_csv = telecharger_fichier_sftp(experiment_name,"mesures",hostName)
print(pathFile_csv)

Connexion SFTP établie avec 169.254.29.18
Téléchargé: mesure_antenne_aimant.csv
Connexion SFTP fermée
mesures/mesures-20251003_133207\mesure_antenne_aimant.csv


In [33]:
time_array, voltage_array_matrix, voltageAcc_array = open_file(pathFile_csv)
plot_acc_only(experiment_name, time_array, voltage_array_matrix, voltageAcc_array)
plot_fourier_transform(time_array, voltageAcc_array)


Fichier mesures/mesures-20251003_133207\mesure_antenne_aimant.csv lu. 10 signaux FID chargés.


C:\Users\page\AppData\Local\Temp\ipykernel_25636\1491225570.py:313: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


Open file bloc

In [38]:
voltage_acc = []

with open(pathFile_csv, 'r', encoding='utf-8') as fichier_:
    lecteur = csv.reader(fichier_)

    # Lire et parser l'entête du fichier
    ligne_entete = next(lecteur)
    dsize = int(ligne_entete[0])
    decimation = int(ligne_entete[1])
    nombre_de_FID = 50                                        #nombre_de_FID = int(ligne_entete[2])
    gain = float(ligne_entete[3])
    offset = float(ligne_entete[4])
    nb_bits = int(ligne_entete[5])

    # Initialisation
    voltage = [[] for _ in range(nombre_de_FID)]
    mean = []
    voltage_acc = np.zeros(dsize)  # Initialiser au bon format et taille

    # Lecture et conversion des tensions
    for j in range(nombre_de_FID):
        ligne = next(lecteur)

        signal = []

        # for val_bin in ligne:
        #     #val_bin = int(val_bin)
        #     val = convert_to_volt(val_bin, nb_bits, gain, offset)
        #     signal.append(val)  # pas besoin de np.append ici


        for val in ligne:
            signal.append(float(val))

        # Convertir la ligne en tableau numpy
        signal = np.array(signal)

        #### Calcul de la moyenne et centrage
        #moyenne = np.mean(signal)
        #signal_centre = signal - moyenne

        # Stocker les données
        voltage[j] = signal
        #mean.append(moyenne)

        # Accumulation du signal total
        voltage_acc += signal

# Création du tableau temps
duree_mesure = (dsize * decimation) / SAMPLING_RATE 
time = np.linspace(0, duree_mesure, dsize, endpoint=False)

# Calcul de la moyenn et centrage du signal accumulé
moyenne = np.mean(voltage_acc[-100000:-1000])
voltage_acc = voltage_acc - moyenne

print(f"Fichier {pathFile_csv} lu. {len(voltage)} signaux FID chargés.")

time_array =  time
voltage_array_matrix = voltage
voltageAcc_array = voltage_acc

Fichier mesures/mesures-20251002_132043\mesure_antenne_petite3.csv lu. 50 signaux FID chargés.


In [60]:
plot_acc_only(experiment_name, time_array, voltage_array_matrix, voltageAcc_array)


In [50]:
plot_fourier_transform(time_array, voltageAcc_array)

C:\Users\page\AppData\Local\Temp\ipykernel_23660\1875004204.py:312: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


Bloc filtrage

In [23]:
# Paramètres du signal
fs = SAMPLING_RATE/decimation  # Fréquence d'échantillonnage (Hz)
t = time_array

freq_basse = 500   # Fréquence basse (Hz)
freq_haute = 10000  # Fréquence haute (Hz)
ordre = 5         # Ordre du filtre

butter = signal.butter(ordre,[freq_basse,freq_haute], 
                    btype='bandpass', fs=fs, output='sos')

signal_filtre = signal.sosfilt(butter, voltageAcc_array)


plt.figure(figsize=(10, 4))
plt.plot(time_array, signal_filtre)
plt.title("Signal filtre")
plt.xlabel("Temps [s]")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

plot_fourier_transform(time_array, signal_filtre)

""" print("Récupération du tableau voltage_acc")
print("Filtrage, Affichage de la FID et de la TF")

b, a = signal.butter(ordre, freq_haute, 
                    btype='lowpass',analog=True)
w, h = signal.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [rad/s]')
plt.ylabel('Amplitude [dB]')
plt.grid(which='both', axis='both')
plt.axvline(freq_haute, color='green') # cutoff frequency
plt.show() """



C:\Users\page\AppData\Local\Temp\ipykernel_25636\1491225570.py:313: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


' print("Récupération du tableau voltage_acc")\nprint("Filtrage, Affichage de la FID et de la TF")\n\nb, a = signal.butter(ordre, freq_haute, \n                    btype=\'lowpass\',analog=True)\nw, h = signal.freqs(b, a)\nplt.semilogx(w, 20 * np.log10(abs(h)))\nplt.title(\'Butterworth filter frequency response\')\nplt.xlabel(\'Frequency [rad/s]\')\nplt.ylabel(\'Amplitude [dB]\')\nplt.grid(which=\'both\', axis=\'both\')\nplt.axvline(freq_haute, color=\'green\') # cutoff frequency\nplt.show() '

In [51]:
plot_fourier_transform(time_array, signal_filtre)


C:\Users\page\AppData\Local\Temp\ipykernel_23660\1875004204.py:312: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


In [122]:
time_array, voltage_array_matrix, voltageAcc_array = open_file(pathFile_csv)


Fichier mesures/mesures-20250930_163654\mesure_antenne_vpetite.csv lu. 100 signaux FID chargés.


In [123]:
plot_acc(experiment_name, time_array, voltage_array_matrix, voltageAcc_array)
plot_fourier_transform(time_array, voltageAcc_array)

C:\Users\page\AppData\Local\Temp\ipykernel_24264\1892533562.py:239: UserWarning: First parameter to grid() is false, but line properties are supplied. The grid will be enabled.
  plt.grid(False, which='both', linestyle='--', linewidth=0.5, alpha=0.7)


## Plot previous acquisiton

In [ ]:
dossier = mesures/mesures-20250829_165839/
nomDossier = "mesures/mesures-20251001_154439/"
nomFichier_local = "mesure_test.csv"

pathFile_csv_new = nomDossier +'/'+ nomFichier_local
#pathFile_csv_new = pathFile_csv



In [15]:
plot_acc_only(nomFichier_local, time_array, voltage_array_matrix, voltageAcc_array)
plot_fourier_transform(time_array, voltageAcc_array)


C:\Users\page\AppData\Local\Temp\ipykernel_34532\3219310908.py:261: UserWarning: First parameter to grid() is false, but line properties are supplied. The grid will be enabled.
  plt.grid(False, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
C:\Users\page\AppData\Local\Temp\ipykernel_34532\3219310908.py:312: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


___

# Balayage temps d'excitation et frequence
Balayage Excitation :

In [49]:
hostName = "rp-f0d66d.local"
#===== PARAMETERS ======

sample_Amount = 1216             #Number of points measured (MAX = 524288) must be a multiple of 2
decimation = 1024                 #Decimation
experiment_name = "balayage_time.csv"            #name of the file of the measured tension
folder_name = "balayage"
larmor_Frequency_Hertz = 12e+3          #Larmor frequency fixed while the excitation time sweep
Start_excitation_duration_seconds = 10.022e-3     #Excitation time start
Nb_of_step = 4               #Nubmer of acquisitons
Step_excitation_time_seconds = 10e-3
Wait_time_microseconds = 2000 + (Step_excitation_time_seconds+4*Step_excitation_time_seconds)*10**6

run_find_excitation_time_command(hostName, sample_Amount, decimation, 
                            Nb_of_step, experiment_name, larmor_Frequency_Hertz, 
                            Start_excitation_duration_seconds, Step_excitation_time_seconds, Wait_time_microseconds)

pathFile_csv = telecharger_fichier_sftp(experiment_name,folder_name,hostName)
time_array, voltage_array_matrix, voltageAcc_array = open_file(pathFile_csv)

amountFID = len(voltage_array_matrix)

plt.figure(figsize=(12, 7))

# Affichage des courbes FID
for w in range(amountFID):
    plt.plot(time_array, voltage_array_matrix[w], marker='+', linestyle='-', label=f'FID {w+1}')


# Mise en forme du graphique
plt.title(f'Superposition - {folder_name}/{experiment_name}')
plt.xlabel('Temps (s)')
plt.ylabel('Tension (V)')
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
plt.legend(loc='upper right')
plt.tight_layout()
#plt.show()


[ERREUR] SSH : [Errno 11001] getaddrinfo failed


UnboundLocalError: cannot access local variable 'command' where it is not associated with a value

Balayage fréquence :

In [ ]:
hostName = "rp-f09fd8.local"
#===== PARAMETERS ======

sample_Amount = 20000            #Number of points measured (MAX = 524288) must be a multiple of 2
decimation = 1024                #Decimation
experiment_name = ".csv"            #name of the file of the measured tension
folder_name = ""
excitation_duration_seconds = 44.022e-6     #Excitation time start
start_larmor_Frequency_Hertz = 24348000      #Larmor frequency fixed while the excitation time sweep
Nb_of_step = 50              #Nubmer of acquisitons
Step_frequency_hertz = 1e+3
Step_excitation_time_seconds = 10e-3
Wait_time_microseconds = 3e+6 + (Step_excitation_time_seconds+4*Step_excitation_time_seconds)*10**6

run_find_frequency_command(hostName, sample_Amount, decimation, 
                            Nb_of_step, experiment_name, start_larmor_Frequency_Hertz, 
                            excitation_duration_seconds, Step_frequency_hertz, Wait_time_microseconds)



In [ ]:

pathFile_csv = telecharger_fichier_sftp(experiment_name,folder_name,hostName)
time_array, voltage_array_matrix, voltageAcc_array = open_file(pathFile_csv)

amountFID = len(voltage_array_matrix)

plt.figure(figsize=(12, 7))

# Affichage des courbes FID
for w in range(amountFID):
    plt.plot(time_array, voltage_array_matrix[w], marker='+', linestyle='-', label=f'FID {w+1}')


# Mise en forme du graphique
plt.title(f'Superposition - {folder_name}/{experiment_name}')
plt.xlabel('Temps (s)')
plt.ylabel('Tension (V)')
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()